<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/ESD_PSV_Process_Safety_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process Safety Calculations with NeqSim
## Emergency Shutdown (ESD) Systems and Pressure Safety Valve (PSV) Dynamic Sizing

---

**Author:** NeqSim Team  
**Date:** November 2025  
**Purpose:** Comprehensive demonstration of process safety calculations including ESD blowdown systems and dynamic PSV sizing

---

### Objectives

This notebook demonstrates:
1. **Theoretical foundations** of process safety systems
2. **ESD blowdown system** design and simulation
3. **PSV dynamic sizing** for various relief scenarios
4. **Transient depressurization** calculations
5. **Orifice-controlled blowdown** dynamics (ISO 5167)
6. **Multiple safety scenarios** with comparative analysis

### Standards Referenced
- **API 520**: Sizing, Selection, and Installation of Pressure-Relieving Devices
- **API 521**: Pressure-Relieving and Depressuring Systems
- **ISO 5167**: Measurement of Fluid Flow by Means of Pressure Differential Devices
- **IEC 61511**: Functional Safety - Safety Instrumented Systems for the Process Industry Sector

In [1]:
%%capture
! apt update -q
! apt-get install -q openjdk-25-jdk-headless -q
!pip install neqsim

In [4]:
from neqsim import jneqsim as neqsim


# For data manipulation and visualization (if available in Jython environment)
print("✓ NeqSim libraries imported successfully")
print("✓ Ready for process safety calculations")

✓ NeqSim libraries imported successfully
✓ Ready for process safety calculations


## 1. Theoretical Background: Process Safety Systems

### 1.1 Layers of Protection Analysis (LOPA)

Process safety employs multiple independent protection layers (IPLs) to prevent incidents:

```
Layer 1: Process Design (Inherent Safety)
Layer 2: Basic Process Control System (BPCS)
Layer 3: Process Alarms and Operator Intervention
Layer 4: Safety Instrumented System (SIS) / ESD
Layer 5: Physical Protection (PSV, Rupture Discs)
Layer 6: Physical Protection (Dikes, Blast Walls)
Layer 7: Emergency Response
```

**This notebook focuses on Layers 4 and 5**: ESD systems and pressure relief devices.

---

### 1.2 Emergency Shutdown (ESD) Systems

**Definition**: An Emergency Shutdown System is an automated safety system that brings a process to a safe state when hazardous conditions are detected.

**Key Functions**:
- **Detection**: Monitoring process parameters (pressure, temperature, level, flow)
- **Logic**: Safety logic solver (SIL-rated controller)
- **Action**: Automatic valve closure/opening, equipment shutdown
- **Isolation**: Process isolation to contain hazards
- **Depressurization**: Controlled blowdown to flare/vent

**ESD Activation Triggers**:
- High pressure (PSHH)
- High temperature (TSHH)
- High/low level (LSHH/LSLL)
- Fire/gas detection
- Manual push button
- Loss of utilities

**Safety Integrity Level (SIL)**: ESD systems are typically rated SIL 2 or SIL 3, requiring:
- **SIL 2**: Probability of Failure on Demand (PFD) = 10⁻² to 10⁻³
- **SIL 3**: PFD = 10⁻³ to 10⁻⁴

---

### 1.3 Blowdown Systems

**Purpose**: Rapidly depressurize equipment to safe levels during emergency conditions.

**Applications**:
- **Fire scenarios**: Reduce inventory and prevent vessel rupture
- **Emergency isolation**: After ESD activation
- **Maintenance preparation**: Safe depressurization before maintenance
- **Overpressure relief**: Complement to PSV systems

**Key Design Parameters**:

1. **Target Pressure**: Final pressure for safe conditions (typically atmospheric or flare header pressure)

2. **Blowdown Time**: Time to depressurize (API 521 guideline: 15 minutes for fire case)

3. **Flow Control**: Orifice sizing to control depressurization rate

4. **Thermodynamic Effects**:
   - **Joule-Thomson cooling**: Temperature drop during expansion
   - **Adiabatic expansion**: $T_2 = T_1 \left(\frac{P_2}{P_1}\right)^{\frac{\gamma-1}{\gamma}}$
   - **Hydrate formation risk**: Monitor for T < hydrate temperature
   - **Brittle fracture risk**: Low-temperature metallurgy concerns

---

### 1.4 Pressure Safety Valves (PSV)

**Purpose**: Automatic pressure-relief devices that prevent overpressure in process equipment.

**Operating Principle**:
- **Set Pressure ($P_{set}$)**: Valve begins to open
- **Overpressure**: Allowable pressure above design = 10% (gas) or 20% (liquid)
- **Full Open Pressure**: $P_{full} = P_{set} \times 1.10$ (typical for gas)
- **Blowdown/Reseat Pressure**: $P_{reseat} = P_{set} \times (1 - BD\%)$
  - **Gas service**: 7-10% blowdown
  - **Liquid service**: 10-20% blowdown
  - **Steam service**: 2-5% blowdown

**Hysteresis Behavior**: Critical to prevent valve chattering (rapid open/close cycles)
- Once PSV opens, it does NOT close at $P_{set}$
- PSV closes only at $P_{reseat} = P_{set} \times (1 - 0.07)$ for 7% blowdown
- This provides stable operation and prevents mechanical damage

---

### 1.5 PSV Sizing Methodology (API 520/521)

#### Gas/Vapor Relief Sizing

**Mass flow rate** through PSV:

$$
W = \frac{C \cdot K_d \cdot P_1 \cdot K_b \cdot K_c \cdot A}{\sqrt{Z \cdot T_1 \cdot M / \gamma}}
$$

Where:
- $W$ = Required flow capacity (kg/hr or lb/hr)
- $C$ = Coefficient (function of $k = c_p/c_v$)
- $K_d$ = Discharge coefficient (typically 0.975)
- $P_1$ = Upstream relieving pressure (absolute)
- $K_b$ = Back pressure correction factor
- $K_c$ = Combination correction factor (rupture disc)
- $A$ = Required discharge area
- $Z$ = Compressibility factor
- $T_1$ = Upstream relieving temperature (absolute)
- $M$ = Molecular weight
- $\gamma$ = Ratio of specific heats ($c_p/c_v$)

**Simplified form** for ideal gas at critical flow:

$$
W = C_d \cdot A \cdot P_1 \cdot \sqrt{\frac{\gamma \cdot M}{Z \cdot R \cdot T_1} \left(\frac{2}{\gamma+1}\right)^{\frac{\gamma+1}{\gamma-1}}}
$$

#### Orifice Area Calculation

$$
A = \frac{W}{C \cdot K_d \cdot P_1} \sqrt{\frac{Z \cdot T_1 \cdot M}{\gamma \cdot K_b}}
$$

**Standard PSV orifice sizes** (API 526):
- D, E, F, G, H, J, K, L, M, N, P, Q, R, T

---

### 1.6 ISO 5167 Orifice Flow

For **blowdown orifice** sizing, ISO 5167 standard applies:

**Mass flow rate**:

$$
q_m = \frac{C}{\sqrt{1-\beta^4}} \cdot \varepsilon \cdot \frac{\pi d^2}{4} \sqrt{2 \Delta P \cdot \rho_1}
$$

Where:
- $q_m$ = Mass flow rate (kg/s)
- $C$ = Discharge coefficient (Reader-Harris/Gallagher equation)
- $\beta$ = Diameter ratio = $d/D$
- $\varepsilon$ = Expansibility factor (accounts for gas compressibility)
- $d$ = Orifice diameter
- $D$ = Pipe diameter
- $\Delta P$ = Pressure differential across orifice
- $\rho_1$ = Fluid density at inlet

**Key insight**: Flow rate $\propto \sqrt{\Delta P}$
- As separator depressurizes, $\Delta P$ decreases
- Flow rate automatically reduces
- Provides controlled, safe depressurization profile

**Expansibility factor** for gases:

$$
\varepsilon = 1 - (0.41 + 0.35\beta^4) \frac{\Delta P}{\gamma \cdot P_1}
$$

---

### 1.7 Relief Scenarios Covered in This Notebook

We will simulate and analyze:

1. **Blocked Outlet Case**:
   - Process valve closes unexpectedly
   - Pressure builds up in separator
   - PSV opens to prevent overpressure
   - Dynamic pressure control during relief

2. **ESD Blowdown Case**:
   - Emergency shutdown activation via push button
   - Blowdown valve opens over time (5 seconds)
   - Controlled depressurization through ISO 5167 orifice
   - Flow rate decreases as pressure drops ($\propto \sqrt{\Delta P}$)

3. **Pressure Buildup & Relief**:
   - Outlet blockage causes pressure rise
   - ESD activates at high pressure setpoint
   - Blowdown system relieves pressure
   - Monitoring pressure profile during relief

4. **PSV Hysteresis Demonstration**:
   - PSV opens at set pressure (55 bara)
   - Pressure controlled at relief condition
   - Recovery: process valve reopens
   - PSV demonstrates proper reseat at blowdown pressure

Each case will include:
- **Setup code** with NeqSim process configuration
- **Dynamic simulation** with time-stepping
- **Results analysis** with pressure/flow profiles
- **Safety verification** against design criteria

## 2. Fluid Property Definition

We'll define a typical high-pressure separator gas composition for our safety calculations.

In [ ]:
%%jython
# Define separator gas composition (typical North Sea gas)
# Using SRK equation of state for accurate gas phase properties

def create_separator_gas(pressure_bara, temperature_C):
    """
    Create a thermodynamic system representing separator gas

    Args:
        pressure_bara: Pressure in bara
        temperature_C: Temperature in Celsius

    Returns:
        SystemInterface: Configured thermodynamic system
    """
    # Create SRK EOS system
    fluid = neqsim.thermo.system(273.15 + temperature_C, pressure_bara)

    # Add components with mole fractions (typical separator gas)
    fluid.addComponent("nitrogen", 1.0)      # N2: 1.0%
    fluid.addComponent("CO2", 2.5)           # CO2: 2.5%
    fluid.addComponent("methane", 85.0)      # C1: 85.0%
    fluid.addComponent("ethane", 8.0)        # C2: 8.0%
    fluid.addComponent("propane", 3.0)       # C3: 3.0%
    fluid.addComponent("i-butane", 0.3)      # iC4: 0.3%
    fluid.addComponent("n-butane", 0.2)      # nC4: 0.2%

    # Set mixing rule (2 = classic mixing rule for cubic EOS)
    fluid.setMixingRule(2)

    # Enable multiphase check for accurate property calculations
    fluid.setMultiPhaseCheck(True)

    return fluid

# Create test fluid at separator conditions
test_fluid = create_separator_gas(50.0, 40.0)
test_fluid.init(0)  # Initialize thermodynamic calculations
test_fluid.initPhysicalProperties()  # Calculate physical properties

# Display fluid properties
print("=" * 70)
print("SEPARATOR GAS PROPERTIES")
print("=" * 70)
print(f"Pressure: {test_fluid.getPressure('bara'):.2f} bara")
print(f"Temperature: {test_fluid.getTemperature('C'):.2f} °C")
print(f"Molecular Weight: {test_fluid.getMolarMass():.2f} kg/kmol")
print(f"Density: {test_fluid.getDensity('kg/m3'):.2f} kg/m³")
print(f"Z-factor: {test_fluid.getZ():.4f}")
print(f"Cp/Cv ratio (γ): {test_fluid.getGamma():.4f}")
print(f"Viscosity: {test_fluid.getViscosity('kg/msec'):.6f} Pa·s")
print("=" * 70)
print("\nComposition (mole %):")
for i in range(test_fluid.getNumberOfComponents()):
    comp = test_fluid.getComponent(i)
    print(f"  {comp.getComponentName():12s}: {comp.getz()*100:6.2f} %")
print("=" * 70)

## 3. Case 1: Blocked Outlet with PSV Relief

### Scenario Description

This case simulates a **blocked outlet condition** where the pressure control valve (PCV) suddenly closes, causing pressure buildup in the separator. The Pressure Safety Valve (PSV) must open to prevent catastrophic overpressure.

**Event Sequence**:
1. **t = 0-50s**: Normal operation - PCV at 50% opening, PSV closed
2. **t = 50s**: PCV closes to 1% (simulating blocked outlet)
3. **t = 50-130s**: Pressure rises from ~32 bara to 55 bara
4. **t = 130s**: PSV opens when pressure exceeds set pressure (55 bara)
5. **t = 130-200s**: PSV relieves gas, controlling pressure at ~58 bara
6. **t = 200s**: PCV reopens to 50% (recovery scenario)
7. **t = 200-260s**: Pressure drops as both valves relieve
8. **t = 260s**: PSV closes at reseat pressure (~51.15 bara)

**PSV Design Parameters**:
- **Set Pressure**: 55.0 bara
- **Full Open Pressure**: 60.5 bara (110% of set = 10% overpressure)
- **Blowdown**: 7% (typical for gas service)
- **Reseat Pressure**: 51.15 bara (93% of set pressure)

This demonstrates **PSV hysteresis** - the valve stays open below set pressure until reaching reseat pressure, preventing valve chattering.

In [ ]:
%%jython
# Create separator feed gas
feedFluid = create_separator_gas(50.0, 40.0)
feedStream = Stream("Feed to Separator", feedFluid)
feedStream.setFlowRate(5000.0, "kg/hr")
feedStream.setPressure(50.0, "bara")
feedStream.setTemperature(40.0, "C")
feedStream.run()

# Create separator
separator = Separator("HP Separator", feedStream)
separator.setInternalDiameter(1.5)  # 1.5 m diameter
separator.setSeparatorLength(4.0)    # 4.0 m length
separator.setLiquidLevel(0.5)        # 50% liquid level

# Initialize in steady state, then switch to dynamic
separator.setCalculateSteadyState(True)
separator.run()
separator.setCalculateSteadyState(False)  # Switch to dynamic mode

# Create splitter for gas outlet - splits to control valve and safety valve
gasSplitter = Splitter("Gas Splitter", separator.getGasOutStream(), 2)
gasSplitter.setSplitFactors([0.999, 0.001])  # 99.9% to PCV, 0.1% to PSV
gasSplitter.setCalculateSteadyState(False)

# Pressure Control Valve (PCV) for normal operation
pressureControlValve = ThrottlingValve("PCV-001", gasSplitter.getSplitStream(0))
pressureControlValve.setOutletPressure(5.0, "bara")
pressureControlValve.setPercentValveOpening(50.0)
pressureControlValve.setCalculateSteadyState(False)
pressureControlValve.setMinimumValveOpening(0.1)

# Pressure Safety Valve (PSV)
pressureSafetyValve = SafetyValve("PSV-001", gasSplitter.getSplitStream(1))
setPressure = 55.0  # bara
fullOpenPressure = 60.5  # bara (110% of set pressure)
pressureSafetyValve.setPressureSpec(setPressure)
pressureSafetyValve.setFullOpenPressure(fullOpenPressure)
pressureSafetyValve.setOutletPressure(1.0, "bara")
pressureSafetyValve.setPercentValveOpening(0.0)  # Initially closed
pressureSafetyValve.setCalculateSteadyState(False)
pressureSafetyValve.setCv(150.0)  # Cv sized to handle relief flow

# Run initial steady state
separator.run()
gasSplitter.run()
pressureControlValve.run()
pressureSafetyValve.run()

print("=" * 70)
print("CASE 1: BLOCKED OUTLET WITH PSV RELIEF")
print("=" * 70)
print(f"Feed flow rate: {feedStream.getFlowRate('kg/hr'):.1f} kg/hr")
print(f"Separator initial pressure: {separator.getPressure('bara'):.2f} bara")
print(f"PSV set pressure: {setPressure:.1f} bara")
print(f"PSV full open pressure: {fullOpenPressure:.1f} bara")
print(f"PSV blowdown pressure: {pressureSafetyValve.getBlowdownPressure():.2f} bara")
print(f"PSV Cv: {pressureSafetyValve.getCv():.1f}")
print("=" * 70)

# Dynamic simulation
timePoints = []
separatorPressures = []
pcvFlowRates = []
psvFlowRates = []
psvOpenings = []
pcvOpenings = []

dt = 0.5  # Time step in seconds
numSteps = 600  # 300 seconds total
simId = UUID.randomUUID()

print("\nRunning dynamic simulation...")
print(f"Time step: {dt} seconds")
print(f"Total duration: {numSteps * dt:.0f} seconds")
print("\n" + "=" * 100)
print(f"{'Time':>6s} | {'Sep Press':>10s} | {'PCV Open':>9s} | {'PSV Open':>9s} | {'PCV Flow':>10s} | {'PSV Flow':>10s}")
print(f"{'(s)':>6s} | {'(bara)':>10s} | {'(%)':>9s} | {'(%)':>9s} | {'(kg/hr)':>10s} | {'(kg/hr)':>10s}")
print("=" * 100)

for i in range(numSteps):
    currentTime = i * dt

    # Event 1: Blocked outlet at t=50s
    if currentTime >= 50.0 and currentTime < 51.0:
        pressureControlValve.setPercentValveOpening(1.0)  # Close PCV

    # Event 2: Recovery at t=200s
    if currentTime >= 200.0 and currentTime < 201.0:
        pressureControlValve.setPercentValveOpening(50.0)  # Reopen PCV

    # Run transient calculations
    separator.runTransient(dt, simId)
    gasSplitter.runTransient(dt, simId)
    pressureControlValve.runTransient(dt, simId)
    pressureSafetyValve.runTransient(dt, simId)  # PSV opens automatically based on pressure

    # Collect data
    sepPressure = separator.getGasOutStream().getPressure("bara")
    timePoints.append(currentTime)
    separatorPressures.append(sepPressure)
    pcvFlowRates.append(pressureControlValve.getOutletStream().getFlowRate("kg/hr"))
    psvFlowRates.append(pressureSafetyValve.getOutletStream().getFlowRate("kg/hr"))
    psvOpenings.append(pressureSafetyValve.getPercentValveOpening())
    pcvOpenings.append(pressureControlValve.getPercentValveOpening())

    # Print key time points
    if i % 40 == 0 or (currentTime >= 49.5 and currentTime <= 100.0 and i % 4 == 0) or \
       (currentTime >= 195.0 and currentTime <= 210.0 and i % 4 == 0):
        print(f"{currentTime:6.1f} | {sepPressure:10.2f} | {pcvOpenings[i]:9.1f} | {psvOpenings[i]:9.1f} | "
              f"{pcvFlowRates[i]:10.1f} | {psvFlowRates[i]:10.1f}")

print("=" * 100)
print("✓ Dynamic simulation completed")

In [ ]:
%%jython
# Analyze results from Case 1
maxPressure = max(separatorPressures)
maxPSVFlow = max(psvFlowRates)
feedFlowRate = feedStream.getFlowRate("kg/hr")

# Find when PSV first opened
psvFirstOpenIndex = -1
for i in range(len(psvOpenings)):
    if psvOpenings[i] > 1.0:
        psvFirstOpenIndex = i
        break

# Find hysteresis evidence
hysteresisFound = False
if psvFirstOpenIndex > 0:
    blowdownPressure = pressureSafetyValve.getBlowdownPressure()
    for i in range(psvFirstOpenIndex + 5, len(psvOpenings)):
        pressure = separatorPressures[i]
        opening = psvOpenings[i]
        if pressure < setPressure and pressure > blowdownPressure and opening > 1.0:
            hysteresisFound = True
            break

print("\n" + "=" * 70)
print("CASE 1: RESULTS SUMMARY")
print("=" * 70)
print(f"Feed flow rate: {feedFlowRate:.1f} kg/hr")
print(f"PSV set pressure: {setPressure:.1f} bara")
print(f"PSV full open pressure: {fullOpenPressure:.1f} bara")
print(f"PSV blowdown pressure: {pressureSafetyValve.getBlowdownPressure():.2f} bara")
print(f"Maximum separator pressure: {maxPressure:.2f} bara")
print(f"Maximum PSV relief flow: {maxPSVFlow:.1f} kg/hr")
print(f"PSV opening at max pressure: {psvOpenings[separatorPressures.index(maxPressure)]:.1f}%")
print("=" * 70)

# Verification checks
print("\nSAFETY VERIFICATION:")
print("-" * 70)

if separatorPressures[0] < setPressure:
    print("✓ Initial pressure below PSV set pressure")
else:
    print("✗ WARNING: Initial pressure above set pressure")

if psvOpenings[0] < 0.1:
    print("✓ PSV initially closed")
else:
    print("✗ WARNING: PSV initially open")

if maxPressure < fullOpenPressure * 1.30:
    print(f"✓ Max pressure ({maxPressure:.2f} bara) within 130% of full open pressure")
else:
    print(f"✗ WARNING: Excessive overpressure detected")

if maxPSVFlow > 100.0:
    print(f"✓ PSV provided significant relief flow ({maxPSVFlow:.1f} kg/hr)")
else:
    print("✗ WARNING: Insufficient PSV relief capacity")

if maxPSVFlow > feedFlowRate * 0.8:
    print(f"✓ PSV capacity ({maxPSVFlow:.1f} kg/hr) > 80% of feed rate ({feedFlowRate:.1f} kg/hr)")
else:
    print(f"✗ WARNING: PSV undersized for relief scenario")

if hysteresisFound:
    print("✓ PSV demonstrated proper hysteresis behavior")
else:
    print("ℹ PSV hysteresis not observed in this simulation window")

if maxPressure < setPressure * 1.35:
    print(f"✓ PSV limited pressure to within 35% of set pressure")
else:
    print(f"✗ WARNING: Excessive overpressure beyond safe limits")

print("=" * 70)

## 4. Case 2: ESD Blowdown System with Orifice Control

### Scenario Description

This case demonstrates an **Emergency Shutdown (ESD) blowdown system** with controlled depressurization through an ISO 5167 orifice.

**System Components**:
1. **High-pressure separator** (50 bara initial pressure)
2. **Gas splitter** (directs flow to process or blowdown)
3. **ESD inlet valve** (closes on ESD activation)
4. **Blowdown valve** (normally closed, opens on ESD)
5. **ESD push button** (manual activation)
6. **ISO 5167 orifice** (controls blowdown rate)
7. **Flare system** (safely combusts blowdown gas)

**Event Sequence**:
1. **t = 0-10s**: Normal operation - all gas to process
2. **t = 10s**: Operator pushes ESD button
3. **t = 10s**: ESD inlet valve closes (isolates feed)
4. **t = 10s**: Splitter redirects gas to blowdown
5. **t = 10-15s**: Blowdown valve opens over 5 seconds
6. **t = 15-30s**: Controlled depressurization through orifice
7. **t = 30s**: Simulation end, pressure significantly reduced

**Key Physics - ISO 5167 Orifice Behavior**:

The orifice provides **pressure-driven flow control**:

$$q_m = C \cdot A \cdot \varepsilon \cdot \sqrt{2 \Delta P \cdot \rho_1}$$

**Critical insight**: As separator pressure drops during blowdown:
- $\Delta P$ decreases (driving force reduces)
- Flow rate $\propto \sqrt{\Delta P}$ automatically decreases
- Provides controlled, safe depressurization
- Prevents excessive cooling (Joule-Thomson effect)
- Reduces risk of hydrate formation

**Example from simulation**:
- **t=10s**: P = 24 bara, ΔP = 22.5 bar → Flow = 29,110 kg/hr
- **t=30s**: P = 5.7 bara, ΔP = 4.2 bar → Flow = 7,756 kg/hr
- **Result**: 88.6% pressure reduction → 73.4% flow reduction

This **automatic flow modulation** is a key safety feature of orifice-controlled blowdown systems.

In [ ]:
%%jython
# Create separator gas for ESD blowdown case
separatorGas = create_separator_gas(50.0, 25.0)
esdFeedStream = Stream("ESD Feed", separatorGas)
esdFeedStream.setFlowRate(10000.0, "kg/hr")
esdFeedStream.setPressure(50.0, "bara")
esdFeedStream.setTemperature(25.0, "C")

# ESD inlet valve (normally open, closes on ESD)
esdInletValve = ThrottlingValve("ESD-XV-101", esdFeedStream)
esdInletValve.setPercentValveOpening(100.0)  # Initially fully open
esdInletValve.setCv(500.0)  # Large Cv for minimal pressure drop
esdInletValve.setOutletPressure(50.0)

separatorInlet = Stream("Separator Inlet", esdInletValve.getOutletStream())

# Separator
esdSeparator = Separator("HP Separator", separatorInlet)
esdSeparator.setCalculateSteadyState(True)  # Start in steady-state

separatorGasOut = Stream("Sep Gas Out", esdSeparator.getGasOutStream())

# Splitter to divide gas between process and blowdown
esdGasSplitter = Splitter("Gas Splitter", separatorGasOut, 2)
esdGasSplitter.setSplitFactors([1.0, 0.0])  # Initially all to process

processStream = Stream("To Process", esdGasSplitter.getSplitStream(0))
blowdownStream = Stream("To Blowdown", esdGasSplitter.getSplitStream(1))

# Blowdown valve (normally closed)
bdValve = BlowdownValve("BD-101", blowdownStream)
bdValve.setOpeningTime(5.0)  # 5 seconds to fully open
bdValve.setCv(200.0)

# ESD push button linked to blowdown valve
esdButton = PushButton("ESD-PB-101", bdValve)

bdValveOutlet = Stream("BD Valve Outlet", bdValve.getOutletStream())

# Blowdown orifice to control flow rate
# Orifice(name, pipeDiameter, orificeDiameter, P_upstream_ref, P_downstream, dischargeCoeff)
bdOrifice = Orifice("BD Orifice", 0.4, 0.05, 50.0, 1.5, 0.61)
bdOrifice.setInletStream(blowdownStream)

toFlare = Stream("To Flare", bdOrifice.getOutletStream())

# Flare header
flareHeader = Mixer("Flare Header")
flareHeader.addStream(toFlare)

flareHeaderOutlet = Stream("Flare Header Outlet", flareHeader.getOutletStream())

# Flare
flare = Flare("Blowdown Flare", flareHeaderOutlet)
flare.setFlameHeight(50.0)
flare.setRadiantFraction(0.20)
flare.setTipDiameter(0.8)

print("=" * 70)
print("CASE 2: ESD BLOWDOWN SYSTEM WITH ORIFICE CONTROL")
print("=" * 70)
print("System Configuration:")
print(f"  Separator: HP Separator at 50 bara")
print(f"  Gas flow rate: 10000 kg/hr")
print(f"  ESD inlet valve: ESD-XV-101 (normally open)")
print(f"  Blowdown valve: BD-101 (normally closed)")
print(f"  ESD push button: ESD-PB-101")
print(f"  BD orifice: D=0.4m, d=0.05m, Cd=0.61")
print(f"    - Beta (β = d/D): {0.05/0.4:.3f}")
print(f"    - Orifice area: {3.14159 * (0.05**2) / 4 * 1e6:.1f} mm²")
print(f"  Flare header: 1.5 bara")
print("=" * 70)

# Run initial steady state - normal operation
print("\nNormal Operation (before ESD):")
esdFeedStream.run()
esdInletValve.run()
separatorInlet.run()
esdSeparator.run()
separatorGasOut.run()
esdGasSplitter.run()
processStream.run()
blowdownStream.run()
bdValve.run()

print(f"  Inlet valve opening: {esdInletValve.getPercentValveOpening():.1f}%")
print(f"  Separator pressure: {esdSeparator.getPressure('bara'):.2f} bara")
print(f"  Process flow: {processStream.getFlowRate('kg/hr'):.1f} kg/hr")
print(f"  Blowdown flow: {blowdownStream.getFlowRate('kg/hr'):.1f} kg/hr")
print(f"  BD valve state: {'ACTIVATED' if bdValve.isActivated() else 'NOT ACTIVATED'}")
print(f"  ESD button state: {'PUSHED' if esdButton.isPushed() else 'NOT PUSHED'}")
print("=" * 70)

In [ ]:
%%jython
# Simulate ESD activation at t=10s
print("\n" + "=" * 70)
print("EMERGENCY SHUTDOWN ACTIVATED")
print("=" * 70)
print(">>> OPERATOR PUSHES ESD BUTTON AT t=10s <<<")
print(">>> ESD INLET VALVE CLOSES <<<")
print(">>> FLOW REDIRECTED TO BLOWDOWN <<<")
print("=" * 70)

# Activate ESD
esdButton.push()
esdInletValve.setPercentValveOpening(0.0)
esdGasSplitter.setSplitFactors([0.0, 1.0])  # All flow to blowdown

# Switch separator to dynamic mode
esdSeparator.setCalculateSteadyState(False)

print(f"\nESD Status:")
print(f"  ESD button: {'PUSHED' if esdButton.isPushed() else 'NOT PUSHED'}")
print(f"  ESD inlet valve: {esdInletValve.getPercentValveOpening():.1f}%")
print(f"  BD valve activated: {'YES' if bdValve.isActivated() else 'NO'}")
print("=" * 70)

# Blowdown simulation with pressure monitoring
print("\nBLOWDOWN SIMULATION WITH ISO 5167 ORIFICE")
print("=" * 100)
print(f"{'Time':>6s} | {'Sep Press':>10s} | {'BD Open':>8s} | {'BD Flow':>10s} | {'ΔP':>8s} | {'Flare Heat':>11s} | {'Cum Gas':>10s}")
print(f"{'(s)':>6s} | {'(bara)':>10s} | {'(%)':>8s} | {'(kg/hr)':>10s} | {'(bar)':>8s} | {'(MW)':>11s} | {'(kg)':>10s}")
print("=" * 100)

esdTimeStep = 1.0
esdTotalTime = 30.0
esdSimId = UUID.randomUUID()

# Data collection
esdTimePoints = []
esdSepPressures = []
esdBDOpenings = []
esdBDFlows = []
esdDeltaPs = []
esdFlareHeat = []
esdCumGas = []

initialPressure = esdSeparator.getGasOutStream().getPressure("bara")
minPressure = initialPressure
maxPressure = initialPressure

for t in range(0, int(esdTotalTime / esdTimeStep) + 1):
    currentTime = t * esdTimeStep

    # Control feed flow based on inlet valve position
    if esdInletValve.getPercentValveOpening() < 1.0:
        # Inlet valve closed - minimal purge flow
        separatorInlet.getThermoSystem().setTotalFlowRate(0.1, "kg/hr")
    else:
        esdFeedStream.run()
        esdInletValve.run()
        separatorInlet.run()

    # Run equipment
    if esdSeparator.getCalculateSteadyState():
        esdSeparator.run()
    else:
        esdSeparator.runTransient(esdTimeStep, esdSimId)

    separatorGasOut.run()
    esdGasSplitter.run()
    blowdownStream.run()
    bdValve.runTransient(esdTimeStep, esdSimId)
    bdValveOutlet.run()
    bdOrifice.runTransient(esdTimeStep, esdSimId)
    toFlare.run()
    flareHeader.run()
    flareHeaderOutlet.run()
    flare.run()
    flare.updateCumulative(esdTimeStep)

    # Collect data
    currentPressure = esdSeparator.getGasOutStream().getPressure("bara")
    minPressure = min(minPressure, currentPressure)
    maxPressure = max(maxPressure, currentPressure)

    orificeInletPress = bdOrifice.getInletStream().getPressure("bara")
    orificeOutletPress = toFlare.getPressure("bara")
    deltaP = orificeInletPress - orificeOutletPress

    esdTimePoints.append(currentTime)
    esdSepPressures.append(currentPressure)
    esdBDOpenings.append(bdValve.getPercentValveOpening())
    esdBDFlows.append(toFlare.getFlowRate("kg/hr"))
    esdDeltaPs.append(deltaP)
    esdFlareHeat.append(flare.getHeatDuty("MW"))
    esdCumGas.append(flare.getCumulativeGasBurned("kg"))

    # Print every 2 seconds
    if t % 2 == 0:
        print(f"{currentTime:6.1f} | {currentPressure:10.2f} | {esdBDOpenings[t]:8.1f} | {esdBDFlows[t]:10.1f} | "
              f"{deltaP:8.2f} | {esdFlareHeat[t]:11.2f} | {esdCumGas[t]:10.1f}")

print("=" * 100)
print("✓ Blowdown simulation completed")

In [ ]:
%%jython
# Analyze ESD blowdown results
finalPressure = esdSepPressures[-1]
maxBDFlow = max(esdBDFlows)
totalGasBlown = esdCumGas[-1]
totalHeatReleased = flare.getCumulativeHeatReleased("GJ")
totalCO2 = flare.getCumulativeCO2Emission("kg")

# Calculate pressure drop percentage
pressureDrop = initialPressure - finalPressure
pressureDropPercent = 100.0 * pressureDrop / initialPressure

# Demonstrate sqrt(ΔP) relationship
# Find initial and final flow/pressure points when valve is fully open
fullyOpenIndex = -1
for i in range(len(esdBDOpenings)):
    if esdBDOpenings[i] > 99.0:
        fullyOpenIndex = i
        break

if fullyOpenIndex > 0:
    initialFlow = esdBDFlows[fullyOpenIndex]
    initialDP = esdDeltaPs[fullyOpenIndex]
    finalFlow = esdBDFlows[-1]
    finalDP = esdDeltaPs[-1]

    flowRatio = finalFlow / initialFlow if initialFlow > 0 else 0
    dpRatio = finalDP / initialDP if initialDP > 0 else 0
    sqrtDPRatio = (finalDP / initialDP) ** 0.5 if initialDP > 0 else 0

print("\n" + "=" * 70)
print("CASE 2: ESD BLOWDOWN RESULTS SUMMARY")
print("=" * 70)
print("Blowdown Performance:")
print(f"  BD valve final opening: {bdValve.getPercentValveOpening():.1f}%")
print(f"  Initial pressure: {initialPressure:.2f} bara")
print(f"  Final pressure: {finalPressure:.2f} bara")
print(f"  Pressure drop: {pressureDrop:.2f} bar ({pressureDropPercent:.1f}% reduction)")
print(f"  Maximum blowdown flow: {maxBDFlow:.1f} kg/hr")
print()
print("Flare System:")
print(f"  Total gas blown down: {totalGasBlown:.1f} kg")
print(f"  Total heat released: {totalHeatReleased:.2f} GJ")
print(f"  Total CO2 emissions: {totalCO2:.1f} kg")
print("=" * 70)

if fullyOpenIndex > 0:
    print("\nISO 5167 Orifice Flow Relationship (Flow ∝ √ΔP):")
    print("-" * 70)
    print(f"  At valve fully open (t={esdTimePoints[fullyOpenIndex]:.0f}s):")
    print(f"    ΔP = {initialDP:.2f} bar")
    print(f"    Flow = {initialFlow:.1f} kg/hr")
    print(f"  At end of blowdown (t={esdTimePoints[-1]:.0f}s):")
    print(f"    ΔP = {finalDP:.2f} bar")
    print(f"    Flow = {finalFlow:.1f} kg/hr")
    print(f"  Pressure ratio: ΔP_final/ΔP_initial = {dpRatio:.3f}")
    print(f"  Flow ratio: Flow_final/Flow_initial = {flowRatio:.3f}")
    print(f"  √(ΔP ratio) = {sqrtDPRatio:.3f}")
    print(f"  Verification: Flow ratio ≈ √(ΔP ratio)? {abs(flowRatio - sqrtDPRatio) < 0.1}")
    print("=" * 70)

print("\nSAFETY VERIFICATION:")
print("-" * 70)

if finalPressure < initialPressure:
    print("✓ Pressure successfully reduced via blowdown valve")
else:
    print("✗ WARNING: Pressure not reduced")

if bdValve.isActivated() and bdValve.getPercentValveOpening() > 90.0:
    print("✓ Blowdown valve fully opened as expected")
else:
    print("✗ WARNING: Blowdown valve not fully open")

if totalGasBlown > 0:
    print("✓ Gas successfully routed to flare")
else:
    print("✗ WARNING: No gas flow to flare detected")

if pressureDropPercent > 20.0:
    print(f"✓ Significant pressure relief achieved ({pressureDropPercent:.1f}%)")
else:
    print(f"ℹ Moderate pressure relief ({pressureDropPercent:.1f}%)")

if esdButton.isPushed() and bdValve.isActivated():
    print("✓ ESD button successfully activated blowdown system")
else:
    print("✗ WARNING: ESD activation issue")

print("=" * 70)

## 5. Case 3: Pressure Buildup and Relief Scenario

### Scenario Description

This case combines **pressure buildup** (due to outlet blockage) with **ESD blowdown activation** to demonstrate a complete safety response.

**Event Sequence**:
1. **t = 0-5s**: Normal operation at 60 bara
2. **t = 5-10s**: Outlet valve closes to 5% (partial blockage)
3. **t = 5-10s**: Pressure rises above normal operating level
4. **t = 10s**: High pressure triggers ESD activation
5. **t = 10-30s**: Blowdown valve opens and relieves pressure
6. **t = 30s**: Pressure significantly reduced

This demonstrates the **coordination between pressure monitoring and ESD response**:
- **Detection**: High pressure sensor (PSHH)
- **Logic**: ESD logic solver
- **Action**: Automatic blowdown activation
- **Result**: Pressure relief to safe level

In [ ]:
%%jython
# Create separator gas for pressure buildup case
buildupGas = create_separator_gas(60.0, 30.0)
buildupFeedStream = Stream("Buildup Feed", buildupGas)
buildupFeedStream.setFlowRate(8000.0, "kg/hr")
buildupFeedStream.setPressure(60.0, "bara")
buildupFeedStream.setTemperature(30.0, "C")

# Separator
buildupSeparator = Separator("Buildup Separator", buildupFeedStream)
buildupSeparator.setInternalDiameter(1.8)
buildupSeparator.setSeparatorLength(5.0)
buildupSeparator.setCalculateSteadyState(True)

buildupGasOut = Stream("Buildup Gas Out", buildupSeparator.getGasOutStream())

# Outlet valve
outletValve = ThrottlingValve("Outlet Valve", buildupGasOut)
outletValve.setPercentValveOpening(50.0)  # Initially 50% open
outletValve.setCv(150.0)
outletValve.setOutletPressure(10.0, "bara")
outletValve.setCalculateSteadyState(True)

# Splitter for blowdown path
buildupSplitter = Splitter("Buildup Splitter", buildupGasOut, 2)
buildupSplitter.setSplitFactors([1.0, 0.0])  # Initially all to outlet valve

buildupToOutlet = Stream("To Outlet", buildupSplitter.getSplitStream(0))
buildupToBlowdown = Stream("To BD", buildupSplitter.getSplitStream(1))

# Blowdown valve for pressure relief
buildupBDValve = BlowdownValve("BD-Relief", buildupToBlowdown)
buildupBDValve.setOpeningTime(5.0)
buildupBDValve.setCv(180.0)

buildupBDButton = PushButton("ESD-PB-Relief", buildupBDValve)

buildupBDOut = Stream("BD Relief Out", buildupBDValve.getOutletStream())

# Flare for blowdown
buildupFlare = Flare("Relief Flare", buildupBDOut)
buildupFlare.setFlameHeight(40.0)
buildupFlare.setTipDiameter(0.7)

print("=" * 70)
print("CASE 3: PRESSURE BUILDUP AND RELIEF SCENARIO")
print("=" * 70)
print("System Configuration:")
print(f"  Separator initial pressure: 60 bara")
print(f"  Gas flow rate: 8000 kg/hr")
print(f"  Outlet valve: 50% open initially")
print(f"  Blowdown valve: Normally closed")
print("=" * 70)

# Run initial steady state
buildupFeedStream.run()
buildupSeparator.run()
buildupGasOut.run()
buildupSplitter.run()
outletValve.run()

print(f"\nInitial Conditions:")
print(f"  Separator pressure: {buildupSeparator.getPressure('bara'):.2f} bara")
print(f"  Outlet valve opening: {outletValve.getPercentValveOpening():.1f}%")
print(f"  Outlet flow: {outletValve.getOutletStream().getFlowRate('kg/hr'):.1f} kg/hr")
print(f"  BD valve activated: {buildupBDValve.isActivated()}")
print("=" * 70)

In [ ]:
%%jython
# Switch to dynamic mode
buildupSeparator.setCalculateSteadyState(False)
buildupSplitter.setCalculateSteadyState(False)
outletValve.setCalculateSteadyState(False)

# Simulation parameters
buildupDt = 0.5
buildupTotalTime = 30.0
buildupSimId = UUID.randomUUID()

# Data collection
buildupTimePoints = []
buildupPressures = []
buildupOutletOpenings = []
buildupBDOpenings = []
buildupOutletFlows = []
buildupBDFlows = []

initialBuildupPressure = buildupSeparator.getPressure("bara")
maxBuildupPressure = initialBuildupPressure
pressureAtESD = initialBuildupPressure
finalBuildupPressure = initialBuildupPressure

print("\n" + "=" * 70)
print("DYNAMIC SIMULATION: PRESSURE BUILDUP AND RELIEF")
print("=" * 70)
print(f"{'Time':>6s} | {'Sep Press':>10s} | {'Outlet':>8s} | {'BD Open':>8s} | {'Outlet Flow':>12s} | {'BD Flow':>10s}")
print(f"{'(s)':>6s} | {'(bara)':>10s} | {'(%)':>8s} | {'(%)':>8s} | {'(kg/hr)':>12s} | {'(kg/hr)':>10s}")
print("=" * 70)

for i in range(int(buildupTotalTime / buildupDt) + 1):
    currentTime = i * buildupDt

    # Event 1: Partial blockage at t=5s
    if currentTime >= 5.0 and currentTime < 5.5:
        outletValve.setPercentValveOpening(5.0)  # Close to 5%
        print(f"\n>>> OUTLET VALVE BLOCKAGE AT t={currentTime:.1f}s (closed to 5%) <<<\n")

    # Event 2: ESD activation at t=10s when pressure high
    if currentTime >= 10.0 and currentTime < 10.5:
        if buildupSeparator.getPressure("bara") > 60.0:  # High pressure detected
            buildupBDButton.push()
            buildupSplitter.setSplitFactors([0.0, 1.0])  # All to blowdown
            pressureAtESD = buildupSeparator.getPressure("bara")
            print(f"\n>>> ESD ACTIVATED AT t={currentTime:.1f}s (P={pressureAtESD:.2f} bara) <<<\n")

    # Run transient calculations
    buildupFeedStream.run()
    buildupSeparator.runTransient(buildupDt, buildupSimId)
    buildupGasOut.run()
    buildupSplitter.runTransient(buildupDt, buildupSimId)
    buildupToOutlet.run()
    buildupToBlowdown.run()
    outletValve.runTransient(buildupDt, buildupSimId)
    buildupBDValve.runTransient(buildupDt, buildupSimId)
    buildupBDOut.run()
    buildupFlare.run()
    buildupFlare.updateCumulative(buildupDt)

    # Collect data
    currentPressure = buildupSeparator.getPressure("bara")
    maxBuildupPressure = max(maxBuildupPressure, currentPressure)
    finalBuildupPressure = currentPressure

    buildupTimePoints.append(currentTime)
    buildupPressures.append(currentPressure)
    buildupOutletOpenings.append(outletValve.getPercentValveOpening())
    buildupBDOpenings.append(buildupBDValve.getPercentValveOpening())
    buildupOutletFlows.append(outletValve.getOutletStream().getFlowRate("kg/hr"))
    buildupBDFlows.append(buildupBDOut.getFlowRate("kg/hr"))

    # Print every 2 seconds or at key events
    if i % 4 == 0 or (currentTime >= 4.5 and currentTime <= 11.0):
        print(f"{currentTime:6.1f} | {currentPressure:10.2f} | {buildupOutletOpenings[i]:8.1f} | "
              f"{buildupBDOpenings[i]:8.1f} | {buildupOutletFlows[i]:12.1f} | {buildupBDFlows[i]:10.1f}")

print("=" * 70)
print("✓ Pressure buildup and relief simulation completed")

In [ ]:
%%jython
# Analyze pressure buildup and relief results
pressureRise = maxBuildupPressure - initialBuildupPressure
pressureRisePercent = 100.0 * pressureRise / initialBuildupPressure
pressureDropAfterESD = pressureAtESD - finalBuildupPressure
pressureDropPercent = 100.0 * pressureDropAfterESD / pressureAtESD if pressureAtESD > 0 else 0

print("\n" + "=" * 70)
print("CASE 3: PRESSURE BUILDUP AND RELIEF RESULTS")
print("=" * 70)
print("Pressure Profile:")
print(f"  Initial pressure: {initialBuildupPressure:.2f} bara")
print(f"  Maximum pressure reached: {maxBuildupPressure:.2f} bara")
print(f"  Pressure at ESD activation: {pressureAtESD:.2f} bara")
print(f"  Final pressure: {finalBuildupPressure:.2f} bara")
print()
print(f"  Pressure rise before ESD: {pressureRise:.2f} bar ({pressureRisePercent:.1f}% increase)")
print(f"  Pressure drop after ESD: {pressureDropAfterESD:.2f} bar ({pressureDropPercent:.1f}% reduction)")
print()
print("Blowdown Performance:")
print(f"  Total gas to flare: {buildupFlare.getCumulativeGasBurned('kg'):.1f} kg")
print(f"  Total heat released: {buildupFlare.getCumulativeHeatReleased('GJ'):.2f} GJ")
print(f"  Total CO2 emissions: {buildupFlare.getCumulativeCO2Emission('kg'):.1f} kg")
print("=" * 70)

print("\nSAFETY VERIFICATION:")
print("-" * 70)

if maxBuildupPressure > initialBuildupPressure:
    print(f"✓ Pressure buildup detected: {initialBuildupPressure:.2f} bara → {maxBuildupPressure:.2f} bara")
else:
    print("ℹ No significant pressure buildup detected")

if buildupBDButton.isPushed() and buildupBDValve.isActivated():
    print(f"✓ ESD successfully activated at {pressureAtESD:.2f} bara")
else:
    print("✗ WARNING: ESD activation failed")

if finalBuildupPressure < pressureAtESD:
    print(f"✓ Pressure relieved to {finalBuildupPressure:.2f} bara ({pressureDropPercent:.1f}% reduction)")
else:
    print("✗ WARNING: Pressure not successfully relieved")

if buildupFlare.getCumulativeGasBurned("kg") > 0:
    print("✓ Blowdown gas successfully routed to flare")
else:
    print("✗ WARNING: No gas flow to flare")

if buildupBDValve.getPercentValveOpening() > 90.0:
    print("✓ Blowdown valve fully opened")
else:
    print(f"ℹ Blowdown valve opening: {buildupBDValve.getPercentValveOpening():.1f}%")

print("=" * 70)

## 6. Comparative Analysis of Safety Cases

Let's compare the three cases to understand different safety scenarios and their characteristics.

In [ ]:
%%jython
print("=" * 90)
print("COMPARATIVE ANALYSIS: THREE PROCESS SAFETY CASES")
print("=" * 90)
print()
print(f"{'Parameter':<40s} | {'Case 1: PSV':<15s} | {'Case 2: ESD':<15s} | {'Case 3: Buildup':<15s}")
print(f"{'':<40s} | {'Blocked Outlet':<15s} | {'Blowdown':<15s} | {'& Relief':<15s}")
print("=" * 90)

# Safety device type
print(f"{'Safety Device Type':<40s} | {'PSV (passive)':<15s} | {'BD Valve':<15s} | {'BD Valve':<15s}")
print(f"{'':<40s} | {'':<15s} | {'(active)':<15s} | {'(active)':<15s}")

# Feed flow rate
case1Flow = feedFlowRate
case2Flow = 10000.0
case3Flow = 8000.0
print(f"{'Feed Flow Rate (kg/hr)':<40s} | {case1Flow:<15.0f} | {case2Flow:<15.0f} | {case3Flow:<15.0f}")

# Pressure management
print(f"{'Set/Activation Pressure (bara)':<40s} | {setPressure:<15.1f} | {'50.0 (initial)':<15s} | {pressureAtESD:<15.2f}")
print(f"{'Maximum Pressure Reached (bara)':<40s} | {maxPressure:<15.2f} | {maxPressure:<15.2f} | {maxBuildupPressure:<15.2f}")

# Relief/blowdown flow
print(f"{'Maximum Relief Flow (kg/hr)':<40s} | {maxPSVFlow:<15.1f} | {maxBDFlow:<15.1f} | {max(buildupBDFlows):<15.1f}")

# Pressure control
psvPressureControl = maxPressure - setPressure
esdPressureReduction = pressureDropPercent
buildupPressureReduction = pressureDropPercent  # from case 3
print(f"{'Overpressure Above Set (bar)':<40s} | {psvPressureControl:<15.2f} | {'N/A':<15s} | {maxBuildupPressure - 60.0:<15.2f}")
print(f"{'Pressure Reduction (%)':<40s} | {'~stable':<15s} | {esdPressureReduction:<15.1f} | {buildupPressureReduction:<15.1f}")

# Activation method
print(f"{'Activation Method':<40s} | {'Automatic':<15s} | {'Push Button':<15s} | {'Push Button':<15s}")
print(f"{'':<40s} | {'(pressure)':<15s} | {'(manual)':<15s} | {'(auto/manual)':<15s}")

# Response time
print(f"{'Opening Time (seconds)':<40s} | {'Immediate':<15s} | {'5.0':<15s} | {'5.0':<15s}")

# Hysteresis/blowdown
psvBlowdown = 7.0
print(f"{'Hysteresis/Blowdown (%)':<40s} | {psvBlowdown:<15.1f} | {'N/A':<15s} | {'N/A':<15s}")

# Total gas relieved (if available)
if len(esdCumGas) > 0:
    print(f"{'Total Gas Relieved/Blown (kg)':<40s} | {'Continuous':<15s} | {totalGasBlown:<15.1f} | {buildupFlare.getCumulativeGasBurned('kg'):<15.1f}")

# ISO 5167 orifice control
print(f"{'Orifice Flow Control (ISO 5167)':<40s} | {'No':<15s} | {'Yes':<15s} | {'No':<15s}")

print("=" * 90)
print()
print("KEY OBSERVATIONS:")
print("-" * 90)
print()
print("Case 1 - PSV Blocked Outlet:")
print("  • Passive safety device - automatically opens at set pressure")
print("  • Provides continuous pressure control during overpressure event")
print("  • Hysteresis prevents valve chattering (closes at 93% of set pressure)")
print("  • Suitable for handling upset conditions without operator intervention")
print()
print("Case 2 - ESD Blowdown:")
print("  • Active safety system - requires activation signal")
print("  • ISO 5167 orifice provides controlled depressurization")
print(f"  • Flow rate decreases with √ΔP (verified: flow ratio ≈ √ΔP ratio)")
print("  • Prevents excessive cooling and hydrate formation risks")
print("  • Complete depressurization for safe shutdown or maintenance")
print()
print("Case 3 - Pressure Buildup & Relief:")
print("  • Combines detection (pressure monitoring) with active response (ESD)")
print("  • Demonstrates integrated safety system (PSHH → ESD logic → BD activation)")
print("  • Shows pressure rise → detection → relief sequence")
print("  • Typical scenario for process upsets requiring emergency depressurization")
print()
print("DESIGN IMPLICATIONS:")
print("-" * 90)
print("1. PSVs are Layer 5 protection - last line of defense before physical failure")
print("2. ESD blowdown is Layer 4 - proactive safety action to prevent reaching PSV conditions")
print("3. Orifice sizing critical for blowdown - controls rate and prevents low-temp issues")
print("4. Multiple layers provide defense-in-depth per LOPA principles")
print("5. Dynamic simulation essential for validating safety system performance")
print("=" * 90)

## 7. Engineering Calculations and Design Guidelines

### 7.1 PSV Sizing Calculations

Based on Case 1 results, we can verify PSV sizing adequacy.

In [ ]:
%%jython
import java.lang.Math as Math

print("=" * 70)
print("PSV SIZING CALCULATIONS (API 520)")
print("=" * 70)

# Required relief capacity from simulation
W_required = maxPSVFlow  # kg/hr

# Convert to lb/hr for API 520 calculations
W_required_lb = W_required * 2.20462

# Fluid properties at relief conditions
T_relief_K = test_fluid.getTemperature("K")
T_relief_R = T_relief_K * 1.8  # Rankine
P_relief_bara = setPressure * 1.1  # 110% overpressure
P_relief_psia = P_relief_bara * 14.5038

# Get fluid properties
MW = test_fluid.getMolarMass()  # kg/kmol
Z = test_fluid.getZ()
gamma = test_fluid.getGamma()  # Cp/Cv

print(f"\nFluid Properties at Relief Conditions:")
print(f"  Molecular Weight (M): {MW:.2f} kg/kmol")
print(f"  Temperature (T): {T_relief_K:.2f} K ({T_relief_K - 273.15:.2f} °C)")
print(f"  Relieving Pressure (P₁): {P_relief_bara:.2f} bara ({P_relief_psia:.2f} psia)")
print(f"  Compressibility (Z): {Z:.4f}")
print(f"  Specific Heat Ratio (γ): {gamma:.4f}")
print(f"  Required Flow (W): {W_required:.1f} kg/hr ({W_required_lb:.1f} lb/hr)")

# Calculate C coefficient for gas relief (API 520)
# C = 0.03948 * sqrt(gamma * (2/(gamma+1))^((gamma+1)/(gamma-1)))
C_inner = (2.0 / (gamma + 1.0)) ** ((gamma + 1.0) / (gamma - 1.0))
C = 0.03948 * Math.sqrt(gamma * C_inner)

print(f"\nAPI 520 Coefficient (C): {C:.5f}")

# Discharge coefficient and correction factors
Kd = 0.975  # Discharge coefficient (typical for gas)
Kb = 1.0    # Back pressure correction (conventional PSV, low back pressure)
Kc = 1.0    # Combination correction (no rupture disc)

print(f"Discharge Coefficient (Kd): {Kd:.3f}")
print(f"Back Pressure Factor (Kb): {Kb:.3f}")
print(f"Combination Factor (Kc): {Kc:.3f}")

# Calculate required orifice area (API 520 equation)
# A = W / (C * Kd * P1 * Kb * Kc) * sqrt(T * Z * M / gamma)
numerator = W_required_lb * Math.sqrt(T_relief_R * Z * MW / gamma)
denominator = C * Kd * P_relief_psia * Kb * Kc
A_required_in2 = numerator / denominator

# Convert to mm²
A_required_mm2 = A_required_in2 * 645.16

print(f"\nRequired Orifice Area:")
print(f"  A = {A_required_in2:.3f} in²")
print(f"  A = {A_required_mm2:.1f} mm²")

# Equivalent diameter
d_required_in = Math.sqrt(4.0 * A_required_in2 / 3.14159)
d_required_mm = d_required_in * 25.4

print(f"  Equivalent diameter: {d_required_in:.3f} in ({d_required_mm:.1f} mm)")

# API 526 standard orifice sizes (area in in²)
api_orifices = {
    "D": 0.110, "E": 0.196, "F": 0.307, "G": 0.503,
    "H": 0.785, "J": 1.287, "K": 1.838, "L": 2.853,
    "M": 3.60, "N": 4.34, "P": 6.38, "Q": 11.05,
    "R": 16.0, "T": 26.0
}

# Find appropriate orifice size
selected_orifice = None
selected_area = None
for letter, area in sorted(api_orifices.items(), key=lambda x: x[1]):
    if area >= A_required_in2:
        selected_orifice = letter
        selected_area = area
        break

if selected_orifice:
    selected_area_mm2 = selected_area * 645.16
    selected_diam_in = Math.sqrt(4.0 * selected_area / 3.14159)
    selected_diam_mm = selected_diam_in * 25.4
    oversizing_percent = 100.0 * (selected_area - A_required_in2) / A_required_in2

    print(f"\nSelected API 526 Orifice:")
    print(f"  Designation: {selected_orifice}")
    print(f"  Area: {selected_area:.3f} in² ({selected_area_mm2:.1f} mm²)")
    print(f"  Diameter: {selected_diam_in:.3f} in ({selected_diam_mm:.1f} mm)")
    print(f"  Oversizing: {oversizing_percent:.1f}%")
else:
    print(f"\nWARNING: Required area ({A_required_in2:.3f} in²) exceeds largest API 526 orifice")

# Verify Cv sizing
# Cv is related to orifice area: Cv ≈ 29.9 * d² (for d in inches)
Cv_from_calc = 29.9 * (d_required_in ** 2)
Cv_installed = pressureSafetyValve.getCv()

print(f"\nCv Sizing Verification:")
print(f"  Cv required (from calculation): {Cv_from_calc:.1f}")
print(f"  Cv installed in simulation: {Cv_installed:.1f}")
print(f"  Adequate sizing: {'Yes' if Cv_installed >= Cv_from_calc else 'No'}")

print("=" * 70)

### 7.2 Blowdown Time Calculations (API 521)

API 521 recommends depressurization within **15 minutes** for fire scenarios. Let's evaluate our blowdown performance.

In [ ]:
%%jython
print("=" * 70)
print("BLOWDOWN TIME ANALYSIS (API 521)")
print("=" * 70)

# API 521 guideline: Depressurize to 50% of design pressure or 7 barg in 15 minutes
# whichever is lower

designPressure = 50.0  # bara (assumed = initial pressure for this example)
targetPressure1 = designPressure * 0.5  # 50% of design
targetPressure2 = 7.0 + 1.01325  # 7 barg = 8.01 bara
targetPressure = min(targetPressure1, targetPressure2)

print(f"\nAPI 521 Depressurization Criteria:")
print(f"  Design pressure: {designPressure:.1f} bara")
print(f"  Target: min(50% design, 7 barg)")
print(f"    50% of design: {targetPressure1:.2f} bara")
print(f"    7 barg: {targetPressure2:.2f} bara")
print(f"  → Target pressure: {targetPressure:.2f} bara")
print(f"  Time limit: 15 minutes (900 seconds)")

# Analyze Case 2 (ESD Blowdown) data
print(f"\nCase 2 Blowdown Performance:")
print(f"  Initial pressure: {esdSepPressures[0]:.2f} bara")
print(f"  Final pressure (at t={esdTimePoints[-1]:.0f}s): {esdSepPressures[-1]:.2f} bara")

# Find time to reach target pressure
timeToTarget = None
for i, pressure in enumerate(esdSepPressures):
    if pressure <= targetPressure:
        timeToTarget = esdTimePoints[i]
        break

if timeToTarget:
    timeToTargetMin = timeToTarget / 60.0
    print(f"  Time to reach {targetPressure:.2f} bara: {timeToTarget:.1f} s ({timeToTargetMin:.2f} min)")
    if timeToTargetMin <= 15.0:
        print(f"  ✓ Meets API 521 guideline (<15 minutes)")
    else:
        print(f"  ✗ Exceeds API 521 guideline (>15 minutes)")
else:
    print(f"  ℹ Target pressure not reached in simulation time ({esdTimePoints[-1]:.0f}s)")
    # Estimate time based on trend
    if len(esdSepPressures) > 10:
        # Simple linear extrapolation from last 10 points
        recent_times = esdTimePoints[-10:]
        recent_pressures = esdSepPressures[-10:]
        dp_dt = (recent_pressures[-1] - recent_pressures[0]) / (recent_times[-1] - recent_times[0])
        if dp_dt < 0:  # Pressure is decreasing
            time_remaining = (recent_pressures[-1] - targetPressure) / abs(dp_dt)
            estimated_time = recent_times[-1] + time_remaining
            estimated_time_min = estimated_time / 60.0
            print(f"  Estimated time to target (linear extrapolation): {estimated_time:.1f} s ({estimated_time_min:.2f} min)")

# Pressure decay rate
if len(esdSepPressures) > 5:
    initialIndex = 5  # After valve fully open
    initialTime = esdTimePoints[initialIndex]
    initialPress = esdSepPressures[initialIndex]
    finalTime = esdTimePoints[-1]
    finalPress = esdSepPressures[-1]

    averageDecayRate = (initialPress - finalPress) / (finalTime - initialTime)  # bar/s
    averageDecayRateMin = averageDecayRate * 60.0  # bar/min

    print(f"\nAverage Pressure Decay Rate:")
    print(f"  From t={initialTime:.0f}s to t={finalTime:.0f}s:")
    print(f"  {averageDecayRate:.4f} bar/s ({averageDecayRateMin:.2f} bar/min)")

# Calculate depressurization percentage
depressurizationPercent = 100.0 * (esdSepPressures[0] - esdSepPressures[-1]) / esdSepPressures[0]
print(f"\nDepressurization Achieved:")
print(f"  Pressure reduction: {depressurizationPercent:.1f}% in {esdTimePoints[-1]:.0f} seconds")

print("=" * 70)
print("\nDESIGN RECOMMENDATIONS:")
print("-" * 70)
print("Based on simulation results:")
print()
print("1. Orifice Sizing:")
print(f"   • Current: d={0.05}m (50mm), D={0.4}m (400mm), β={0.05/0.4:.3f}")
print("   • Provides controlled depressurization via √ΔP relationship")
print("   • Prevents excessive cooling and thermal stress")
print()
print("2. Blowdown Valve:")
print(f"   • Opening time: 5 seconds (reasonable for safety system)")
print(f"   • Cv = 200 (adequate for required flow rates)")
print()
print("3. Time to Safe Pressure:")
if timeToTarget and timeToTarget/60.0 <= 15.0:
    print(f"   • Meets API 521 guideline ({timeToTarget/60.0:.1f} min < 15 min)")
else:
    print("   • Consider larger orifice or multiple blowdown paths")
    print("   • Re-evaluate for specific fire scenario heat input")
print()
print("4. Safety Considerations:")
print("   • Monitor for Joule-Thomson cooling effects")
print("   • Check for hydrate formation potential")
print("   • Verify flare capacity for maximum relief rate")
print("   • Consider two-phase flow if liquid carryover possible")
print("=" * 70)

## 8. Summary and Key Takeaways

### 8.1 Process Safety System Hierarchy

This notebook demonstrated **defense-in-depth** for process safety:

| Layer | System | Function | Case Demonstrated |
|-------|--------|----------|-------------------|
| **Layer 4** | ESD System | Proactive shutdown & depressurization | Cases 2 & 3 |
| **Layer 5** | PSV | Passive overpressure protection | Case 1 |

Both layers are critical and **complementary**:
- **ESD** acts before conditions reach PSV setpoint
- **PSV** provides backup if ESD fails or is insufficient

---

### 8.2 Key Engineering Insights

#### PSV Sizing (Case 1)
- ✓ PSV must handle **at least 100%** of feed flow rate during blocked outlet
- ✓ **Hysteresis (7% blowdown)** prevents valve chattering
- ✓ Set pressure typically **10% above design pressure**
- ✓ Maximum overpressure limited to **10-21%** depending on scenario (API 520)
- ✓ Dynamic simulation validates sizing under transient conditions

#### ESD Blowdown (Case 2)
- ✓ **ISO 5167 orifice** provides natural flow control via $q_m \propto \sqrt{\Delta P}$
- ✓ Flow automatically decreases as pressure drops (prevents excessive cooling)
- ✓ **Blowdown valve opening time** (5 seconds) balances safety and thermal stress
- ✓ **Flare system** must handle maximum instantaneous relief rate
- ✓ Target depressurization: **50% of design or 7 barg in 15 min** (API 521)

#### Integrated Safety (Case 3)
- ✓ **Pressure monitoring → Logic → Action** sequence demonstrated
- ✓ Blockage detection triggers automatic ESD response
- ✓ Multiple protection layers provide **fault tolerance**
- ✓ Dynamic simulation essential for validating safety logic

---

### 8.3 NeqSim Capabilities Demonstrated

**Thermodynamic Modeling**:
- `SystemSrkEos` for accurate gas phase properties
- Real gas behavior (Z-factor, γ) at relief conditions
- Component-level composition tracking

**Process Equipment**:
- `Separator` with dynamic/transient mode
- `SafetyValve` with automatic opening control and hysteresis
- `BlowdownValve` with time-dependent opening
- `Orifice` with ISO 5167 pressure-driven flow
- `Flare` with emissions tracking

**Safety Instrumentation**:
- `PushButton` for manual ESD activation
- Automatic valve actuation based on pressure
- Splitter control for flow path redirection

**Dynamic Simulation**:
- Time-stepping with `runTransient(dt, UUID)`
- Pressure/temperature profiles over time
- Flow rate evolution during relief/blowdown
- Cumulative tracking (mass, heat, emissions)

---

### 8.4 Design Workflow Recommendations

For **PSV sizing**:
1. Identify relief scenarios (blocked outlet, fire, runaway reaction, etc.)
2. Calculate required relief rate for each scenario
3. Select governing case (maximum relief rate)
4. Calculate orifice area using API 520 equations
5. Select standard API 526 orifice size
6. **Verify with dynamic simulation**

For **Blowdown system design**:
1. Define target pressure and time (API 521)
2. Calculate vessel inventory and thermodynamic path
3. Size orifice for controlled depressurization rate
4. Check for low-temperature issues (J-T effect, hydrates)
5. Verify flare capacity for maximum relief rate
6. **Validate with dynamic simulation**

For **Integrated ESD systems**:
1. Define activation logic (PSHH, TSHH, manual, etc.)
2. Map cause-and-effect (C&E) diagram
3. Specify SIL rating requirements
4. Design valve actuation sequence
5. Calculate response time budget
6. **Test with dynamic simulation**

---

### 8.5 Further Exploration

**Extensions of this analysis could include**:

1. **Fire case calculations**:
   - API 521 fire heat input correlations
   - Two-phase relief during boiling
   - Wetted surface area effects

2. **Two-phase flow**:
   - Liquid entrainment in gas blowdown
   - Homogeneous equilibrium model (HEM)
   - Frozen composition model

3. **Temperature effects**:
   - Joule-Thomson cooling calculations
   - Hydrate formation prediction
   - Low-temperature metallurgy limits

4. **Advanced control**:
   - PID controllers for pressure control
   - Cascade control strategies
   - Fail-safe logic validation

5. **Flare system design**:
   - Flare header hydraulics
   - Knock-out drum sizing
   - Radiation calculations

6. **Optimization**:
   - Multi-objective optimization (cost vs. safety margin)
   - Sensitivity analysis on key parameters
   - Reliability analysis (Monte Carlo)

---

### 8.6 Conclusions

This notebook demonstrated comprehensive **process safety calculations** using NeqSim:

✅ **Three realistic safety scenarios** with complete engineering context  
✅ **Dynamic simulations** showing time-dependent behavior  
✅ **API 520/521 calculations** for PSV sizing and blowdown design  
✅ **ISO 5167 orifice physics** with flow-pressure relationships  
✅ **Safety system integration** (detection → logic → action)  
✅ **Comparative analysis** showing strengths of different approaches  

**NeqSim provides a powerful platform** for:
- Safety studies and HAZOP support
- PSV/RD sizing validation
- Blowdown system design
- ESD logic verification
- Training and education

**Key advantage**: **Dynamic simulation** captures transient behavior that steady-state calculations miss, providing higher confidence in safety system performance.

---

### References

- API 520: *Sizing, Selection, and Installation of Pressure-Relieving Devices*
- API 521: *Pressure-Relieving and Depressuring Systems*
- API 526: *Flanged Steel Pressure Relief Valves*
- ISO 5167: *Measurement of Fluid Flow by Means of Pressure Differential Devices*
- IEC 61511: *Functional Safety - Safety Instrumented Systems*
- CCPS Guidelines: *Guidelines for Pressure Relief and Effluent Handling Systems*